<a href="https://colab.research.google.com/github/shekharkshitij/Deep_Learning/blob/main/DL(Lab05).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

# Function to create the model
def create_model(base_model):
    base_model.trainable = False  # Freeze the base model
    model = tf.keras.Sequential([
        base_model,
        Flatten(),
        Dense(512, activation='relu'),  # Increased dense layer size for more complexity
        Dense(256, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to unfreeze layers
def unfreeze_layers(model, num_layers):
    for layer in model.layers[-num_layers:]:
        layer.trainable = True

# Fine-tuning function
def fine_tune_model(base_model, model_name):
    results = {}
    model = create_model(base_model)
    initial_accuracy = model.evaluate(x_val, y_val, verbose=0)[1]
    results['initial'] = initial_accuracy

    for stage in range(1, 4):
        unfreeze_layers(base_model, stage * 5)  # progressively unfreeze layers
        model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, batch_size=128, verbose=1)  # Increased batch size and epochs
        accuracy = model.evaluate(x_val, y_val, verbose=0)[1]
        results[f'stage_{stage}'] = accuracy

    print(f'{model_name} fine-tuning results:', results)
    return results

# Load pre-trained models
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Perform layer-wise fine-tuning for VGG16 and ResNet50
vgg16_results = fine_tune_model(vgg16_base, "VGG16")
resnet50_results = fine_tune_model(resnet50_base, "ResNet50")

# Final evaluation on the test set
vgg16_base.trainable = True
resnet50_base.trainable = True
vgg16_model = create_model(vgg16_base)
resnet50_model = create_model(resnet50_base)

vgg16_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=128, verbose=1)  # Further increase in epochs
resnet50_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=128, verbose=1)

vgg16_test_acc = vgg16_model.evaluate(x_test, y_test, verbose=0)[1]
resnet50_test_acc = resnet50_model.evaluate(x_test, y_test, verbose=0)[1]

print(f'Final VGG16 test accuracy: {vgg16_test_acc}')
print(f'Final ResNet50 test accuracy: {resnet50_test_acc}')

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 43ms/step - accuracy: 0.3613 - loss: 1.8558 - val_accuracy: 0.6111 - val_loss: 1.1240
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.6387 - loss: 1.0462 - val_accuracy: 0.6701 - val_loss: 0.9592
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.6858 - loss: 0.9085 - val_accuracy: 0.6948 - val_loss: 0.8941
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.7132 - loss: 0.8265 - val_accuracy: 0.7090 - val_loss: 0.8505
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.7328 - loss: 0.7708 - val_accuracy: 0.7144 - val_loss: 0.8286
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 65ms/step - accuracy: 0.7498 - loss: 0.7178 - val_accuracy: 0.7491 - val_loss: 0.7412
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.7955 -

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

# Function to create base models
def create_base_model(base_model, input_shape=(32, 32, 3)):
    base_model.trainable = False
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = Flatten()(x)
    outputs = Dense(10, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load pre-trained models that support 32x32 input size
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Create models from the base
vgg16_model = create_base_model(vgg16_base)
resnet50_model = create_base_model(resnet50_base)
mobilenet_model = create_base_model(mobilenet_base)

# Train each model independently
vgg16_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, batch_size=128, verbose=1)
resnet50_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, batch_size=128, verbose=1)
mobilenet_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, batch_size=128, verbose=1)

# Get predictions from each model
vgg16_predictions = vgg16_model.predict(x_val)
resnet50_predictions = resnet50_model.predict(x_val)
mobilenet_predictions = mobilenet_model.predict(x_val)

# Stacking the predictions
stacked_predictions = np.hstack((vgg16_predictions, resnet50_predictions, mobilenet_predictions))

# Meta-learner
y_val_labels = np.argmax(y_val, axis=1)
meta_learner = LogisticRegression(max_iter=200)
meta_learner.fit(stacked_predictions, y_val_labels)

# Get predictions on the test set for evaluation
vgg16_test_predictions = vgg16_model.predict(x_test)
resnet50_test_predictions = resnet50_model.predict(x_test)
mobilenet_test_predictions = mobilenet_model.predict(x_test)

# Stack test predictions
stacked_test_predictions = np.hstack((vgg16_test_predictions, resnet50_test_predictions, mobilenet_test_predictions))
stacked_test_labels = meta_learner.predict(stacked_test_predictions)

# Evaluate the performance
y_test_labels = np.argmax(y_test, axis=1)
stacked_test_accuracy = accuracy_score(y_test_labels, stacked_test_labels)
print(f'Ensemble Test Accuracy: {stacked_test_accuracy}')

# Individual model performance for comparison
vgg16_test_acc = vgg16_model.evaluate(x_test, y_test, verbose=0)[1]
resnet50_test_acc = resnet50_model.evaluate(x_test, y_test, verbose=0)[1]
mobilenet_test_acc = mobilenet_model.evaluate(x_test, y_test, verbose=0)[1]

print(f'VGG16 Test Accuracy: {vgg16_test_acc}')
print(f'ResNet50 Test Accuracy: {resnet50_test_acc}')
print(f'MobileNetV2 Test Accuracy: {mobilenet_test_acc}')


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


<ipython-input-1-b45b24e2fdce>:39: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.2937 - loss: 2.0227 - val_accuracy: 0.4847 - val_loss: 1.5306
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.4986 - loss: 1.4882 - val_accuracy: 0.5180 - val_loss: 1.4105
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.5308 - loss: 1.3856 - val_accuracy: 0.5335 - val_loss: 1.3530
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.5510 - loss: 1.3257 - val_accuracy: 0.5480 - val_loss: 1.3150
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.5600 - loss: 1.2915 - val_accuracy: 0.5579 - val_loss: 1.2888
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 48ms/step - accuracy: 0.1822 - loss: 2.2322 - val_accuracy: 0.2780 - val_loss: 2.0281
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.2709 - loss: 2.0196 - val_accuracy: 0.3028 - val_loss: 1.9451
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step